## Obtencion del factor temporal de los códigos

In [1]:
from PyPDF2 import PdfReader, PdfWriter

def split_pdf(input_pdf, start_page, end_page, output_pdf):
    reader = PdfReader(input_pdf)
    writer = PdfWriter()

    for i in range(start_page - 1, end_page):
        writer.add_page(reader.pages[i])

    with open(output_pdf, "wb") as out_file:
        writer.write(out_file)

# Ejemplo de uso
split_pdf("../../data\Documentación ficheros de usuario EPF 2006 Año 2021.pdf", 64, 259, "../../data/anexo_codes.pdf")


In [2]:
import pdfplumber
import re

def extract_codes_and_factors(pdf_path):
    keywords = {"Bisemanal", "Mensual", "Trimestral", "Último", "Anual"}  # Factores posibles
    data = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                lines = text.split("\n")  # Dividir en líneas
                
                for i in range(len(lines) - 1):
                    line = lines[i].strip()  # Eliminar espacios y tabulaciones al inicio

                    # Buscar código al inicio de la línea, permitiendo que el último carácter sea una letra
                    match = re.match(r'^(\d{2})\.(\d)\.(\d)\.([\dA-Z])', line)
                    if match:
                        clean_code = "".join(match.groups())  # Unir los números y letras sin puntos

                        # Buscar el factor en las siguientes 6 líneas
                        factor = "NaN"
                        for j in range(1, 7):  # Buscar hasta 6 líneas más abajo
                            if i + j < len(lines):
                                next_line = lines[i + j].strip()  # Eliminar espacios en la siguiente línea
                                first_word = next_line.split()[0] if next_line else ""  # Tomar la primera palabra
                                
                                if first_word in keywords:
                                    factor = first_word
                                    break  # Si ya encontramos un factor, salimos del bucle

                        data.append((clean_code, factor))

    return data


In [3]:
import pandas as pd

# 🔹 Extraer códigos y factores del PDF
pdf_path = "../../data/anexo_codes.pdf"  # Reemplázalo con la ruta correcta
codes_factors = extract_codes_and_factors(pdf_path)

# 🔹 Crear DataFrame y asegurar que `CODIGO` sea string
df = pd.DataFrame(codes_factors, columns=["CODIGO", "FACTOR_TEMPORAL"])
df["CODIGO"] = df["CODIGO"].astype(str)  # Convertir a string por seguridad



In [4]:
for code, factor in codes_factors:
    print(f"{code} -> {factor}")

01111 -> Bisemanal
01112 -> Bisemanal
01113 -> Bisemanal
01114 -> Bisemanal
01115 -> Bisemanal
01116 -> Bisemanal
01117 -> Bisemanal
01118 -> Bisemanal
01121 -> Bisemanal
01122 -> Bisemanal
01123 -> Bisemanal
01124 -> Bisemanal
01125 -> Bisemanal
01126 -> Bisemanal
01127 -> Bisemanal
01128 -> Bisemanal
01131 -> Bisemanal
01132 -> Bisemanal
01133 -> Bisemanal
01134 -> Bisemanal
01135 -> Bisemanal
01136 -> Bisemanal
01141 -> Bisemanal
01142 -> Bisemanal
01143 -> Bisemanal
01144 -> Bisemanal
01145 -> Bisemanal
01146 -> Bisemanal
01147 -> Bisemanal
01151 -> Bisemanal
01152 -> Bisemanal
01153 -> Bisemanal
01154 -> Bisemanal
01155 -> Bisemanal
01161 -> Bisemanal
01162 -> Bisemanal
01163 -> Bisemanal
01164 -> Bisemanal
01165 -> Bisemanal
01166 -> Bisemanal
01167 -> Bisemanal
01168 -> Bisemanal
01169 -> Bisemanal
01160 -> Bisemanal
01171 -> Bisemanal
01172 -> Bisemanal
01173 -> Bisemanal
01174 -> Bisemanal
01175 -> Bisemanal
01176 -> Bisemanal
01177 -> Bisemanal
01178 -> Bisemanal
01179 -> Bis

Manejamos manualmente los NaN

In [5]:
# Quiero ver de df las filas que FACTOR_TEMPORAL sea igual a Nan
print(df[df["FACTOR_TEMPORAL"] == "NaN"])

    CODIGO FACTOR_TEMPORAL
105  04210             NaN
106  04221             NaN
107  04222             NaN
108  04223             NaN
181  04122             NaN
194  06212             NaN
196  06212             NaN
214  04124             NaN
294  10400             NaN
295  10222             NaN
302  10214             NaN
360  09210             NaN


In [6]:
df.loc[df["CODIGO"] == "09210", "FACTOR_TEMPORAL"] = "Anual"
df.loc[df["CODIGO"].astype(str).str.startswith("0422"), "FACTOR_TEMPORAL"] = "Último"
df.loc[df["CODIGO"] == "06212", "FACTOR_TEMPORAL"] = "Trimestral"
df.loc[df["CODIGO"].isin(["10214", "10400", "04124", "04210", "04122", "10222"]), "FACTOR_TEMPORAL"] = "Último"



In [7]:
df = df.drop_duplicates(subset=["CODIGO"])

In [8]:
df[df["FACTOR_TEMPORAL"] == "NaN"]

,CODIGO,FACTOR_TEMPORAL


Comparamos con los codigos de gastos para ver que códigos nos faltan

In [9]:
df_gastos = pd.read_csv("../../DataLake/2023/family_expenses.tsv", sep="\t", encoding="utf-8")

In [10]:
codigos_unicos = df_gastos[~df_gastos["CODIGO"].isin(df["CODIGO"])]
codigos_unicos["CODIGO"].unique()

array(['0117B', '0117A', '0117C'], dtype=object)

In [11]:
nuevos_codigos = pd.DataFrame({
    "CODIGO": ['0117A', '0117B', '0117C'],
    "FACTOR_TEMPORAL": ["Bisemanal"] * 3
})

# Concatenar con el DataFrame existente
df = pd.concat([df, nuevos_codigos], ignore_index=True)


In [12]:
codigos_unicos = df_gastos[~df_gastos["CODIGO"].isin(df["CODIGO"])]
codigos_unicos["CODIGO"].unique()

array([], dtype=object)

Guardamos los códigos con sus factores

In [13]:
# 🔹 Guardar en CSV con separación por tabuladores
df.to_csv("../../data/codes_factors.csv", sep="\t", index=False)

### Creamos un datamart con los gastos agregados por super categorías

Primero cargamos el datalake

In [14]:
import os
import glob
import pandas as pd

In [15]:
def load_datalake(datalake_path):
    """
    Carga los archivos family_expenses.tsv de todos los años dentro del datalake y los concatena en un único DataFrame.
    Añade una columna 'Año' correspondiente al año de cada archivo.
    """
    all_files = glob.glob(os.path.join(datalake_path, "*", "family_expenses.tsv"))
    df_list = []

    for file in all_files:
        year = os.path.basename(os.path.dirname(file))  # Extraer el año de la ruta
        df = pd.read_csv(file, sep="\t")  # Cargar el archivo
        df["Año"] = int(year)  # Añadir la columna Año
        df_list.append(df)

    if not df_list:
        raise ValueError("No se encontraron archivos family_expenses.tsv en la ruta del datalake.")

    # Concatenar todos los DataFrames
    df_expenses = pd.concat(df_list, ignore_index=True)
    return df_expenses


In [16]:
# Ruta del Datalake
datalake_path = "../../Datalake/"  # Ajusta según la ubicación real

# Cargar los datos del datalake
df_expenses = load_datalake(datalake_path)

Rellenar los codigos a 5 digitos

In [17]:
def pad_code(code):
    return str(code).zfill(5)

df_expenses["CODIGO"] = df_expenses["CODIGO"].apply(pad_code)
# Asegurar que los códigos en df_expenses son strings
df_expenses["CODIGO"] = df_expenses["CODIGO"].astype(str)

df_expenses

,NUMERO,CODIGO,GASTOMON,FACTOR,Año
0,1,01112,29415522.00,9.026143e+08,2006
1,1,01114,2494436.00,9.026143e+08,2006
2,1,01115,57113177.00,9.026143e+08,2006
3,1,01117,6000766.00,9.026143e+08,2006
4,1,01121,27085813.00,9.026143e+08,2006
...,...,...,...,...,...
34655650,20707,12521,106228.09,5.447594e+02,2023
34655651,20707,12522,136189.86,5.447594e+02,2023
34655652,20707,12541,163427.83,5.447594e+02,2023
34655653,20707,12620,47712.61,5.447594e+02,2023


In [18]:
def aggregate_supercategories_two_digits(df):
    """
    Agrega los datos por supercategorías, utilizando los dos primeros dígitos del código como categoría.
    Agrupa por AÑO, NUMERO y SUPER_CATEGORIA.
    """
    df["SUPER_CATEGORIA"] = df["CODIGO"].astype(str).str[:2]  # Extraer los dos primeros dígitos del código

    # Agregar los gastos por supercategoría, año y número de hogar
    df_agg = df.groupby(["Año", "NUMERO", "SUPER_CATEGORIA"])["GASTOMON"].sum().reset_index()

    return df_agg

In [19]:
# Agregar por supercategorías
df_aggregated = aggregate_supercategories_two_digits(df_expenses)

# Mostrar el DataFrame con los datos agregados
df_aggregated.head()

,Año,NUMERO,SUPER_CATEGORIA,GASTOMON
0,2006,1,01,684981607.0
1,2006,1,02,319264308.0
2,2006,1,03,34285154.0
3,2006,1,04,175239084.0
4,2006,1,05,503926195.0


In [20]:
df_expenses

,NUMERO,CODIGO,GASTOMON,FACTOR,Año,SUPER_CATEGORIA
0,1,01112,29415522.00,9.026143e+08,2006,01
1,1,01114,2494436.00,9.026143e+08,2006,01
2,1,01115,57113177.00,9.026143e+08,2006,01
3,1,01117,6000766.00,9.026143e+08,2006,01
4,1,01121,27085813.00,9.026143e+08,2006,01
...,...,...,...,...,...,...
34655650,20707,12521,106228.09,5.447594e+02,2023,12
34655651,20707,12522,136189.86,5.447594e+02,2023,12
34655652,20707,12541,163427.83,5.447594e+02,2023,12
34655653,20707,12620,47712.61,5.447594e+02,2023,12


In [21]:
# crear directorio de salida si no existe
output_dir = "../../datamarts"
os.makedirs(output_dir, exist_ok=True)

In [22]:
# Guardar el DataFrame agregado como un archivo TSV
output_filename = "../../datamarts/datamart_supercategories.tsv"
df_aggregated.to_csv(output_filename, sep="\t", index=False)

print(f"Datamart saved as: {output_filename}")


Datamart saved as: ../../datamarts/datamart_supercategories.tsv


In [23]:
def aggregate_supercategories_three_digits(df):
    """
    Agrega los datos por supercategorías, utilizando los dos primeros dígitos del código como categoría.
    Agrupa por AÑO, NUMERO y SUPER_CATEGORIA.
    """
    df["SUPER_CATEGORIA"] = df["CODIGO"].astype(str).str[:3]  # Extraer los dos primeros dígitos del código

    # Agregar los gastos por supercategoría, año y número de hogar
    df_agg = df.groupby(["Año", "NUMERO", "SUPER_CATEGORIA"])["GASTOMON"].sum().reset_index()

    return df_agg

In [24]:
# Agregar por supercategorías
df_aggregated = aggregate_supercategories_three_digits(df_expenses)

# Mostrar el DataFrame con los datos agregados
df_aggregated.head()

,Año,NUMERO,SUPER_CATEGORIA,GASTOMON
0,2006,1,011,684981607.0
1,2006,1,021,319264308.0
2,2006,1,031,34285154.0
3,2006,1,042,0.0
4,2006,1,043,108380640.0


### Ajustar los cambios de las dos encuestas (2006 a 2015) y (2016 a 2023)

In [25]:
import pandas as pd

# 1) Si tu SUPER_CATEGORIA vienen como enteros o sin ceros a la izquierda, pásalos a str de 3 dígitos:
df_aggregated['SUPER_CATEGORIA'] = df_aggregated['SUPER_CATEGORIA'].astype(str).str.zfill(3)

# 2) Definimos el diccionario de mapeo (igual que antes):
mapping = {
    "023": "022",
    "103": "102",
    "104": "103",
    "105": "104",
    "127": "128",
    "126": "127",
    "125": "126",
    "124": "125",
    "123": "124",
    "122": "123"
}

# 3) Aplicamos el mapeo sobre df, no sobre el ya agrupado:
df_aggregated['SUPER_CATEGORIA'] = df_aggregated['SUPER_CATEGORIA'].replace(mapping)

# 4) Ahora agrupas incluyendo NUMERO:
df_aggregated = (
    df_aggregated
    .groupby(['Año', 'NUMERO', 'SUPER_CATEGORIA'], as_index=False)['GASTOMON']
    .sum()
)

# 5) (Opcional) ordena para que sea más legible:
df_aggregated = df_aggregated.sort_values(['Año', 'NUMERO', 'SUPER_CATEGORIA']).reset_index(drop=True)


In [26]:
df_aggregated.head(20)

,Año,NUMERO,SUPER_CATEGORIA,GASTOMON
0,2006,1,011,684981607.0
1,2006,1,021,319264308.0
2,2006,1,031,34285154.0
3,2006,1,042,0.0
4,2006,1,043,108380640.0
5,2006,1,044,22059893.0
6,2006,1,045,44798551.0
7,2006,1,053,293570921.0
8,2006,1,055,59273497.0
9,2006,1,056,151081777.0


In [27]:
# Guardar el DataFrame agregado como un archivo TSV
output_filename = "../../datamarts/datamart_supercategories_three_digits.tsv"
df_aggregated.to_csv(output_filename, sep="\t", index=False)

print(f"Datamart saved as: {output_filename}")


Datamart saved as: ../../datamarts/datamart_supercategories_three_digits.tsv


In [28]:
def aggregate_supercategories_four_digits(df):
    """
    Agrega los datos por supercategorías, utilizando los dos primeros dígitos del código como categoría.
    Agrupa por AÑO, NUMERO y SUPER_CATEGORIA.
    """
    df["SUPER_CATEGORIA"] = df["CODIGO"].astype(str).str[:4]  # Extraer los dos primeros dígitos del código

    # Agregar los gastos por supercategoría, año y número de hogar
    df_agg = df.groupby(["Año", "NUMERO", "SUPER_CATEGORIA"])["GASTOMON"].sum().reset_index()

    return df_agg

In [29]:
# Agregar por supercategorías
df_aggregated_four = aggregate_supercategories_four_digits(df_expenses)

# Mostrar el DataFrame con los datos agregados
df_aggregated_four.head()

,Año,NUMERO,SUPER_CATEGORIA,GASTOMON
0,2006,1,0111,95023901.0
1,2006,1,0112,45558760.0
2,2006,1,0113,45370500.0
3,2006,1,0114,26403373.0
4,2006,1,0116,161667708.0


In [30]:
# Guardar el DataFrame agregado como un archivo TSV
output_filename = "../../datamarts/datamart_supercategories_four_digits.tsv"
df_aggregated_four.to_csv(output_filename, sep="\t", index=False)

print(f"Datamart saved as: {output_filename}")


Datamart saved as: ../../datamarts/datamart_supercategories_four_digits.tsv
